<a href="https://colab.research.google.com/github/mavropulokn/pandasHw/blob/main/pandasHw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from datetime import datetime

import numpy as np
import pandas as pd
from IPython.display import display


class FeatureGenerator:
    def __init__(self, df_train_path, df_questions_path, df_lectures_path, skip_blank_lines=True, nrows=None):
        pd.options.display.max_columns = 100
        self.df_train = pd.read_csv(df_train_path, skip_blank_lines=skip_blank_lines, nrows=nrows)
        self.df_lectures = pd.read_csv(df_lectures_path, skip_blank_lines=skip_blank_lines, nrows=nrows)
        self.df_questions = pd.read_csv(df_questions_path, skip_blank_lines=skip_blank_lines, nrows=nrows)
        self.df_merged = None

    def generate(self):
        self.clear_data()
        self.merge_atomise_dataframes()
        self.analyze_create_features()

    def clear_data(self):
        print(datetime.utcnow(), ": <-- Очистка данных -->")

        """По условию значения df_train prior_question_elapsed_time, user_answer, answered_correctly и
              prior_question_elapsed_time отсутствуют для лекций, поэтому очистим лишь прочие строки df_train"""
        self.df_train = self.df_train[self.df_train['row_id'].notna()
                                      & self.df_train['timestamp'].notna()
                                      & self.df_train['user_id'].notna()
                                      & self.df_train['content_id'].notna()
                                      & self.df_train['content_type_id'].notna()
                                      & self.df_train['task_container_id'].notna()]

        self.df_train = self.clear_data_auxiliary(self.df_train, "df_train", False)
        self.df_questions = self.clear_data_auxiliary(self.df_questions, "df_questions")
        self.df_lectures = self.clear_data_auxiliary(self.df_lectures, "df_lectures")

        print("Выведем первые три строки и посмотрим, насколько полезны столбцы:")
        print("---------------------------------------------------------")
        display(self.df_train.head(3))
        display(self.df_questions.head(3))
        display(self.df_lectures.head(3))
        print("---------------------------------------------------------")

        """Удалим столбец type_of из df_lectures, краткое описание лекции,
              поскольку качественная обработка данных значений 
              может потребовать глубокого использования инструментов NLP,
              например лемматизации или стемматизации"""
        self.df_lectures = self.df_lectures.drop('type_of', axis=1)

        """Удалим столбец user_answer из df_train,
              поскольку нам интересны лишь результаты ответа, а тип интерактивности
              можно вывести df_train content_type_id"""
        self.df_train = self.df_train.drop('user_answer', axis=1)

        """Удалим столбец correct_answer из df_questions,"
              поскольку верный ответ или нет уже известно из df_train answered_correctly"""
        self.df_questions = self.df_questions.drop('correct_answer', axis=1)

        print(datetime.utcnow(), ": <-- Очистка данных завершена -->")

    def merge_atomise_dataframes(self):
        print(datetime.utcnow(), ": <-- Объединение датафреймов в один -->")

        print("Выведем типы для train dataframe: ", self.df_train.dtypes)

        """Объединим датафреймы, добавив к train, преобразуем к атомарности значения столбцов."""
        self.df_merged = self.df_train.merge(self.df_questions, how="left", left_on='content_id',
                                             right_on="question_id")
        self.df_merged = self.df_merged.drop('question_id', axis=1)
        self.df_merged = self.df_merged.rename(columns={'tags': 'tag_question'})
        self.df_train = None
        self.df_questions = None

        self.df_merged = self.df_merged.merge(self.df_lectures, how="left", left_on='content_id',
                                              right_on="lecture_id", suffixes=("", "_lecture"))
        self.df_merged = self.df_merged.drop('lecture_id', axis=1)
        self.df_merged = self.df_merged.rename(columns={'tag': 'tag_lecture'})

        self.df_lectures = None

        print(datetime.utcnow(),
              ": <-- Объединение датафреймов в один -->")

    def analyze_create_features(self):
        print(datetime.utcnow(), ": <-- Анализ объединенного датафрейма и создание фичей -->")
        print("Выведем типы для объединенного dataframe: ", self.df_merged.dtypes)
        print("Выведем размерность для объединенного dataframe: ", self.df_merged.shape)
        print("Выведем описательную статистику для объединенного dataframe: ", self.df_merged.describe())

        print("Выведем первых трех пользователей по числу верных ответов:")
        print(self.df_merged[self.df_merged['content_type_id'] == 0]
              .groupby('user_id')['answered_correctly'].sum().sort_values(ascending=False).head(3))
        print("Выведем первые три пары пользователь - task_container_id по числу верных ответов")
        print(self.df_merged[self.df_merged['content_type_id'] == 0]
              .groupby(['user_id', 'task_container_id'])['answered_correctly'].sum().sort_values(
            ascending=False).head(3))
        print("Выведем первые три пары пользователь - part по числу верных ответов")
        print(self.df_merged[self.df_merged['content_type_id'] == 0]
              .groupby(['user_id', 'part'])['answered_correctly'].sum().sort_values(ascending=False).head(3))
        print("Выведем первые три пары пользователь - tag_question по числу верных ответов")
        print(self.df_merged[self.df_merged['content_type_id'] == 0]
              .groupby(['user_id', 'tag_question'])['answered_correctly'].sum()
              .sort_values(ascending=False).head(3))
        print("Выведем первые три пары пользователь - bundle_id по числу верных ответов")
        print(self.df_merged[self.df_merged['content_type_id'] == 0]
              .groupby(['user_id', 'bundle_id'])['answered_correctly'].sum().sort_values(ascending=False).head(3))

        print("Выведем первых трех пользователей по времени, данному на верные ответы:")
        print(self.df_merged[(self.df_merged['content_type_id'] == 0)
                             & (self.df_merged['answered_correctly'] == 1)]
              .groupby('user_id')['timestamp'].sum().sort_values(ascending=False).head(3))
        print("Выведем первые три пары пользователь - task_container_id по времени, данному на верные ответы")
        print(self.df_merged[(self.df_merged['content_type_id'] == 0)
                             & (self.df_merged['answered_correctly'] == 1)]
              .groupby(['user_id', 'task_container_id'])['timestamp'].sum().sort_values(
            ascending=False).head(3))
        print("Выведем первые три пары пользователь - part по времени, данному на верные ответы")
        print(self.df_merged[(self.df_merged['content_type_id'] == 0)
                             & (self.df_merged['answered_correctly'] == 1)]
              .groupby(['user_id', 'part'])['timestamp'].sum().sort_values(ascending=False).head(3))
        print("Выведем первые три пары пользователь - tag_question по времени, данному на верные ответы")
        print(self.df_merged[(self.df_merged['content_type_id'] == 0)
                             & (self.df_merged['answered_correctly'] == 1)]
              .groupby(['user_id', 'tag_question'])['timestamp'].sum()
              .sort_values(ascending=False).head(3))
        print("Выведем первые три пары пользователь - bundle_id по времени, данному на верные ответы")
        print(self.df_merged[(self.df_merged['content_type_id'] == 0)
                             & (self.df_merged['answered_correctly'] == 1)]
              .groupby(['user_id', 'bundle_id'])['timestamp'].sum().sort_values(ascending=False).head(3))

        print("Выведем первых трех пользователей по времени, затраченному на лекции:")
        print(self.df_merged[self.df_merged['content_type_id'] == 1]
              .groupby('user_id')['timestamp'].sum().sort_values(ascending=False).head(3))
        print("Выведем первые три пары пользователь - tag_lecture по времени, затраченному на лекции")
        print(self.df_merged[self.df_merged['content_type_id'] == 1]
              .groupby(['user_id', 'tag_lecture'])['timestamp'].sum()
              .sort_values(ascending=False).head(3))
        print("Выведем первые три пары пользователь - part_lecture по времени, затраченному на лекции")
        print(self.df_merged[self.df_merged['content_type_id'] == 1]
              .groupby(['user_id', 'part_lecture'])['timestamp'].sum().sort_values(ascending=False).head(3))

        print("Выведем первых трех пользователей по числу просмотра правильных ответов (подсказок):")
        print(self.df_merged[self.df_merged['content_type_id'] == 0]
              .groupby('user_id')['prior_question_had_explanation'].count().sort_values(ascending=False).head(3))

        print("Выведем первых трех пользователей по среднему времени ответов на предыдущие серии вопросов:")
        sum_prior_question_elapset_time_by_user = \
            self.df_merged[self.df_merged['content_type_id'] == 0].groupby('user_id')[
                'prior_question_elapsed_time'].sum().sort_values(ascending=False)
        count_prior_question_elapset_time_by_user = \
            self.df_merged[self.df_merged['content_type_id'] == 0].groupby('user_id')[
                'prior_question_elapsed_time'].count().sort_values(ascending=False)
        sum_prior_question_elapset_time_by_user_count_prior_question_elapset_time_by_user_merged = \
            pd.DataFrame(sum_prior_question_elapset_time_by_user).merge(
                pd.DataFrame(count_prior_question_elapset_time_by_user), how="left", on='user_id')
        sum_prior_question_elapset_time_by_user_count_prior_question_elapset_time_by_user_merged[
            'prior_question_elapsed_time_average'] = \
            sum_prior_question_elapset_time_by_user_count_prior_question_elapset_time_by_user_merged[
                "prior_question_elapsed_time_x"] / \
            sum_prior_question_elapset_time_by_user_count_prior_question_elapset_time_by_user_merged[
                "prior_question_elapsed_time_y"]

        print(sum_prior_question_elapset_time_by_user_count_prior_question_elapset_time_by_user_merged.head(3))

        self.df_merged['tag_question'] = self.df_merged['tag_question'].str.split(" ")
        self.df_merged = self.df_merged.explode('tag_question')  # приведем значения к атомарным

        print("Выведем первые три пары пользователь - tag_question по количеству верных ответов")
        print(self.df_merged[(self.df_merged['content_type_id'] == 0)
                             & (self.df_merged['answered_correctly'] == 1)]
              .groupby(['user_id', 'tag_question'])['tag_question'].count().sort_values(ascending=False).head(3))

        print("Вышеприведенные показатели могут быть использованы как отдельные фичи в рамках статичтического"
              "анализа (например, с использованием матрицы парных корреляций), что за пределами домашней задачи.")

        print(datetime.utcnow(), ": <-- Анализ объединенного датафрейма и создание фичей завершены -->")

    @staticmethod
    def clear_data_auxiliary(data_frame, data_frame_name, dropna=True):
        print(datetime.utcnow(), ": Очищаем данные %s .." % data_frame_name)
        if (dropna):
            print("измерения %s до dropna():" % data_frame_name, data_frame.shape)
            data_frame = data_frame.dropna()
        print("измерения %s до удаления пустых строк:" % data_frame_name, data_frame.shape)
        data_frame = data_frame.replace(r'^\s*$', np.nan, regex=True)
        print("измерения %s после очистки:" % data_frame_name, data_frame.shape)
        return data_frame


In [11]:
train_300000_url = 'https://raw.githubusercontent.com/mavropulokn/pandasHw/main/resources/train_300000.csv'
questions_url = 'https://raw.githubusercontent.com/mavropulokn/pandasHw/main/resources/questions.csv'
lectures_url = 'https://raw.githubusercontent.com/mavropulokn/pandasHw/main/resources/lectures.csv'

generator = FeatureGenerator(train_300000_url,
                            questions_url,
                            lectures_url)
generator.generate()

2022-10-09 20:59:19.993357 : <-- Очистка данных -->
2022-10-09 20:59:20.019672 : Очищаем данные df_train ..
измерения df_train до удаления пустых строк: (300000, 11)
измерения df_train после очистки: (300000, 11)
2022-10-09 20:59:20.342328 : Очищаем данные df_questions ..
измерения df_questions до dropna(): (13523, 5)
измерения df_questions до удаления пустых строк: (13522, 5)
измерения df_questions после очистки: (13522, 5)
2022-10-09 20:59:20.370960 : Очищаем данные df_lectures ..
измерения df_lectures до dropna(): (418, 4)
измерения df_lectures до удаления пустых строк: (418, 4)
измерения df_lectures после очистки: (418, 4)
Выведем первые три строки и посмотрим, насколько полезны столбцы:
---------------------------------------------------------


,Unnamed: 0,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,2,118363,115,128,0,0,0,1,55000.0,False


,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92


,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept


---------------------------------------------------------
2022-10-09 20:59:20.425657 : <-- Очистка данных завершена -->
2022-10-09 20:59:20.426500 : <-- Объединение датафреймов в один -->
Выведем типы для train dataframe:  Unnamed: 0                          int64
row_id                              int64
timestamp                           int64
user_id                             int64
content_id                          int64
content_type_id                     int64
task_container_id                   int64
answered_correctly                  int64
prior_question_elapsed_time       float64
prior_question_had_explanation     object
dtype: object
2022-10-09 20:59:20.632714 : <-- Объединение датафреймов в один -->
2022-10-09 20:59:20.634073 : <-- Анализ объединенного датафрейма и создание фичей -->
Выведем типы для объединенного dataframe:  Unnamed: 0                          int64
row_id                              int64
timestamp                           int64
user_id             